In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
from run_attachment import *

In [ ]:
os.environ['PYOPENGL_PLATFORM'] = 'egl'

In [ ]:
import pyrender

In [ ]:
# Coordinate system transformation (OpenCV cam to OpenGL cam)
cvcam_in_glcam = np.array([[1, 0, 0, 0],
                          [0, -1, 0, 0],
                          [0, 0, -1, 0],
                          [0, 0, 0, 1]])

def render_mesh_at_pose(mesh, pose, K, H, W, zfar=100):
    # Transform mesh to OpenGL camera frame
    mesh_transformed = mesh.copy()
    mesh_transformed.apply_transform(cvcam_in_glcam @ pose)
    
    # Setup pyrender scene
    scene = pyrender.Scene(ambient_light=[1., 1., 1.], bg_color=[0, 0, 0])
    
    # Add camera
    camera = pyrender.IntrinsicsCamera(
        fx=K[0, 0], fy=K[1, 1],
        cx=K[0, 2], cy=K[1, 2],
        znear=0.1, zfar=zfar
    )
    scene.add(camera, pose=np.eye(4))
    
    # Add mesh
    pyrender_mesh = pyrender.Mesh.from_trimesh(mesh_transformed, smooth=False)
    scene.add(pyrender_mesh, pose=np.eye(4))
    
    # Render
    renderer = pyrender.OffscreenRenderer(W, H)
    color, depth = renderer.render(scene)
    renderer.delete()
    
    return color, depth

def visualize_comparison(mesh, pose, scene_dir, i:int=0):
    reader = Ho3dReader(video_dir=scene_dir, root_dir='/Experiments/simonep01/demo_data/light_ho3d')
    rgb = reader.get_color(i)
    
    # Check if mesh is a file path or trimesh object
    if isinstance(mesh, str):
        mesh = trimesh.load(mesh)

    H, W = rgb.shape[:2]
    # Render mesh at gt_pose
    rendered, depth = render_mesh_at_pose(mesh, pose, reader.K, H, W)
    
    # Create visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(rgb)
    axes[0].set_title('Original RGB Image')
    axes[0].axis('off')
    
    # Rendered mesh
    axes[1].imshow(rendered)
    axes[1].set_title('Rendered Mesh at GT Pose')
    axes[1].axis('off')
    
    # Overlay
    overlay = rgb.copy()
    mask = rendered.sum(axis=2) > 0
    overlay[mask] = rendered[mask] * 0.6 + rgb[mask] * 0.4
    axes[2].imshow(overlay)
    axes[2].set_title('Overlay (60% mesh, 40% RGB)')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

## First frame

In [ ]:
mesh_file= '/Experiments/simonep01/demo_data/ho3d_first_frame_instantmeshes/AP14/mesh.obj'
test_scene_dir= '/Experiments/simonep01/demo_data/light_ho3d/evaluation/AP14'
est_refine_iter= 5
track_refine_iter= 2
debug= 0
debug_dir= 'debug'
attach_every_n_frames= 2
boundary_distance_px= 20

set_logging_format()
set_seed(0)

mesh = trimesh.load(mesh_file)
reader = Ho3dReader(video_dir=test_scene_dir)

In [ ]:
mesh_file= '/Experiments/simonep01/demo_data/ho3d_first_frame_instantmeshes/AP14/mesh.obj'
test_scene_dir= '/Experiments/simonep01/demo_data/light_ho3d/evaluation/AP14'
est_refine_iter= 5
track_refine_iter= 2
debug= 0
debug_dir= 'debug'
attach_every_n_frames= 2
gt_mesh_file = '/Experiments/simonep01/demo_data/light_ho3d/models/019_pitcher_base/textured_simple.obj'
boundary_distance_px= 20

set_logging_format()
set_seed(0)

mesh = trimesh.load(mesh_file)

reader = Ho3dReader(video_dir=test_scene_dir)

mesh, _ = estimate_and_scale_mesh(mesh,reader, gt_mesh_file=gt_mesh_file)

CMesh = MeshWithConfidence(mesh)

os.system(f'rm -rf {debug_dir}/* && mkdir -p {debug_dir}/track_vis {debug_dir}/ob_in_cam')

to_origin, extents = trimesh.bounds.oriented_bounds(CMesh.mesh)
bbox = np.stack([-extents/2, extents/2], axis=0).reshape(2,3)

scorer = ScorePredictor()
refiner = PoseRefinePredictor()
glctx = dr.RasterizeCudaContext()
est = FoundationPose(model_pts=CMesh.mesh.vertices, model_normals=CMesh.mesh.vertex_normals, mesh=CMesh.mesh, scorer=scorer, refiner=refiner, debug_dir=debug_dir, debug=debug, glctx=glctx)

i=0
color = reader.get_color(i)
depth = reader.get_depth(i)
mask = reader.get_mask(i).astype(bool)
pose = est.register(K=reader.K, rgb=color, depth=depth, ob_mask=mask, iteration=est_refine_iter)

tmp_mesh = CMesh.mesh.copy()

if attach_every_n_frames > 0 and i % attach_every_n_frames == 0:
        CMesh = perform_attachment(est, CMesh, pose, reader, i)

os.makedirs(f'{debug_dir}/ob_in_cam', exist_ok=True)
np.savetxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt', pose.reshape(4,4))

In [ ]:
CMesh.mesh.show()

In [ ]:
visualize_comparison(tmp_mesh, pose, test_scene_dir, i=i)

In [ ]:
def visualize_confidence_pointcloud(CMesh):
    """Visualize mesh vertices as colored point cloud by confidence
    
    Colors: red (0) → orange (0-0.25) → yellow (0.25-0.5) → blue (0.5-1) → green (1)
    """
    import plotly.graph_objects as go
    
    # Assign colors based on confidence
    colors = np.zeros((len(CMesh.confidence), 3), dtype=np.uint8)
    
    mask_zero = CMesh.confidence == 0
    mask_low = (CMesh.confidence > 0) & (CMesh.confidence < 0.25)
    mask_mid = (CMesh.confidence >= 0.25) & (CMesh.confidence < 0.5)
    mask_high = (CMesh.confidence >= 0.5) & (CMesh.confidence < 1)
    mask_one = CMesh.confidence >= 1.0
    
    colors[mask_zero] = [255, 0, 0]      # Red
    colors[mask_low] = [255, 125, 0]     # Orange
    colors[mask_mid] = [255, 255, 0]     # Yellow
    colors[mask_high] = [0, 0, 255]      # Blue
    colors[mask_one] = [0, 255, 0]       # Green
    
    # Create plotly scatter3d
    fig = go.Figure(data=[go.Scatter3d(
        x=CMesh.observed_positions[:, 0],
        y=CMesh.observed_positions[:, 1],
        z=CMesh.observed_positions[:, 2],
        mode='markers',
        marker=dict(size=2, color=[f'rgb({c[0]},{c[1]},{c[2]})' for c in colors])
    )])
    
    fig.update_layout(
        scene=dict(aspectmode='data'),
        title=f'Confidence: {mask_zero.sum()} red (0), {mask_low.sum()} orange (0-0.25), '
              f'{mask_mid.sum()} yellow (0.25-0.5), {mask_high.sum()} blue (0.5-1), {mask_one.sum()} green (1)'
    )
    
    return fig

# Usage in notebook:
fig = visualize_confidence_pointcloud(CMesh)
fig.show()


## Following Frames

In [ ]:
for j in range(attach_every_n_frames):
    i+=1
    if i < len(reader.color_files):
        color = reader.get_color(i)
        depth = reader.get_depth(i)
        pose = est.track_one(rgb=color, depth=depth, K=reader.K, iteration=track_refine_iter)
        
        tmp_mesh = CMesh.mesh.copy()
        if attach_every_n_frames > 0 and i % attach_every_n_frames == 0:
            CMesh = perform_attachment(est, CMesh, pose, reader, i)
        
        os.makedirs(f'{debug_dir}/ob_in_cam', exist_ok=True)
        np.savetxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt', pose.reshape(4,4))

fig = visualize_confidence_pointcloud(CMesh)
fig.show()

In [ ]:
for j in range(attach_every_n_frames):
    i+=1
    if i < len(reader.color_files):
        color = reader.get_color(i)
        depth = reader.get_depth(i)
        pose = est.track_one(rgb=color, depth=depth, K=reader.K, iteration=track_refine_iter)
        
        tmp_mesh = CMesh.mesh.copy()
        if attach_every_n_frames > 0 and i % attach_every_n_frames == 0:
            CMesh = perform_attachment(est, CMesh, pose, reader, i)
        
        os.makedirs(f'{debug_dir}/ob_in_cam', exist_ok=True)
        np.savetxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt', pose.reshape(4,4))

fig = visualize_confidence_pointcloud(CMesh)
fig.show()

In [ ]:
visualize_comparison(tmp_mesh, pose, test_scene_dir, i=i)

In [ ]:
for j in range(10):
    i+=1
    if i < len(reader.color_files):
        color = reader.get_color(i)
        depth = reader.get_depth(i)
        pose = est.track_one(rgb=color, depth=depth, K=reader.K, iteration=track_refine_iter)
        
        tmp_mesh = CMesh.mesh.copy()
        if attach_every_n_frames > 0 and i % attach_every_n_frames == 0:
            CMesh = perform_attachment(est, CMesh, pose, reader, i)
        
        os.makedirs(f'{debug_dir}/ob_in_cam', exist_ok=True)
        np.savetxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt', pose.reshape(4,4))

fig = visualize_confidence_pointcloud(CMesh)
fig.show()

In [ ]:
visualize_comparison(tmp_mesh, pose, test_scene_dir, i=i)

In [ ]:
for j in range(20):
    i+=1
    if i < len(reader.color_files):
        color = reader.get_color(i)
        depth = reader.get_depth(i)
        pose = est.track_one(rgb=color, depth=depth, K=reader.K, iteration=track_refine_iter)
        
        tmp_mesh = CMesh.mesh.copy()
        if attach_every_n_frames > 0 and i % attach_every_n_frames == 0:
            CMesh = perform_attachment(est, CMesh, pose, reader, i)
        
        os.makedirs(f'{debug_dir}/ob_in_cam', exist_ok=True)
        np.savetxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt', pose.reshape(4,4))

fig = visualize_confidence_pointcloud(CMesh)
fig.show()

In [ ]:
visualize_comparison(tmp_mesh, pose, test_scene_dir, i=i)

In [ ]:
while i < len(reader.color_files)-1:
    i+=1
    color = reader.get_color(i)
    depth = reader.get_depth(i)
    pose = est.track_one(rgb=color, depth=depth, K=reader.K, iteration=track_refine_iter)
    
    tmp_mesh = CMesh.mesh.copy()
    if attach_every_n_frames > 0 and i % attach_every_n_frames == 0:
        CMesh = perform_attachment(est, CMesh, pose, reader, i)
    
    os.makedirs(f'{debug_dir}/ob_in_cam', exist_ok=True)
    np.savetxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt', pose.reshape(4,4))

fig = visualize_confidence_pointcloud(CMesh)
fig.show()

In [ ]:
CMesh.mesh.show()

In [ ]:
visualize_comparison(tmp_mesh, pose, test_scene_dir, i=i)

In [ ]:
CMesh.mesh.export(f'{debug_dir}/attached_mesh.obj')